# Ocean Sciences 

## Homework #4

In [1]:
# imports
import numpy as np
from importlib import reload

from oceanpy import constants
from oceanpy import units
from oceanpy.forces import coriolis as ofc

from bokeh import plotting
from bokeh import models

In [2]:
def set_fontsize(p, fsz):
    '''
    Parameters
    ----------
    ax : Bokeh plot class
    fsz : float
      Font size
    '''
    p.xaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.xaxis.major_label_text_font_size = "{:d}pt".format(fsz)
    #
    p.yaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.yaxis.major_label_text_font_size = "{:d}pt".format(fsz)

# 1 Coriolis Acceleration

## a) Coriolis parameter

### $f = 2 \Omega \, \sin\phi$

### Santa Cruz

In [3]:
f_SC = ofc.coriolis_parameter(37*units.deg)
f_SC

<Quantity 8.77701897e-05 1 / s>

### Range of Latitudes

In [4]:
phis = np.linspace(-90., 90., 180)*units.deg
f = ofc.coriolis_parameter(phis)

In [5]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Coriolis Parameter', x_axis_label='Latitude [deg]',
                   y_axis_label='f [1/s]')

p.line(phis, f, legend='f(phi)', line_width=2)
p.legend.location = "bottom_left"

set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### It is a maximum at +90 deg (90N) and a minimum at -90deg (90S).

### It is zero at 0 deg.

## b) Meridional derivative

### $\beta = \partial f / \partial y = \frac{1}{R} \frac{\partial f}{\partial \phi}$

### $\beta = \frac{2 \Omega}{R}  \, \cos\phi$

In [6]:
beta = ofc.meridional_derivative(phis)
beta

<Quantity [1.40014671e-24, 4.01298220e-10, 8.02472831e-10, 1.20340026e-09,
           1.60395702e-09, 2.00401972e-09, 2.40346514e-09, 2.80217024e-09,
           3.20001220e-09, 3.59686849e-09, 3.99261686e-09, 4.38713542e-09,
           4.78030264e-09, 5.17199742e-09, 5.56209911e-09, 5.95048754e-09,
           6.33704309e-09, 6.72164669e-09, 7.10417987e-09, 7.48452480e-09,
           7.86256433e-09, 8.23818201e-09, 8.61126215e-09, 8.98168982e-09,
           9.34935093e-09, 9.71413223e-09, 1.00759214e-08, 1.04346069e-08,
           1.07900783e-08, 1.11422261e-08, 1.14909419e-08, 1.18361182e-08,
           1.21776487e-08, 1.25154283e-08, 1.28493527e-08, 1.31793193e-08,
           1.35052264e-08, 1.38269736e-08, 1.41444617e-08, 1.44575930e-08,
           1.47662711e-08, 1.50704008e-08, 1.53698885e-08, 1.56646419e-08,
           1.59545703e-08, 1.62395843e-08, 1.65195961e-08, 1.67945195e-08,
           1.70642698e-08, 1.73287640e-08, 1.75879205e-08, 1.78416595e-08,
           1.80899029e-08

In [7]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Meridional derivative', x_axis_label='Latitude [deg]',
                                       y_axis_label='beta [1/km s]')


p.line(phis, beta, legend='beta(phi)', line_width=2)
p.legend.location = "top_left"

set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### $\beta$ is positive everywhere except $\phi = 90$deg where it is zero.

## c) Acceleration

### $a = d^2x/dt^2$

### Integrate once:  $v(t) = at + v_0$

### Integrate twice:  $x(t) = at^2/2 + v_0 t + x_0$

## d) Baseball

### Imperial to SI

In [16]:
v_0 = 80 * units.imperial.mi / units.hour
v_0.cgs.to('km/hr')

<Quantity 128.74752 km / h>

In [15]:
d = 60 * units.imperial.foot
d.cgs.to('m')

<Quantity 18.288 m>

### Coriolis acceleration

### $f v$

In [17]:
# Assume Santa Cruz
f = ofc.coriolis_parameter(37*units.deg)

In [19]:
a_c = f * v_0
a_c.to('m/s**2')

<Quantity 0.00313894 m / s2>

### Time to plate

In [21]:
t = d/v_0
t.to('s')

<Quantity 0.51136364 s>

### Deflection

In [23]:
x = a_c * t**2 / 2
x.to('cm')

<Quantity 0.04104054 cm>

## e) Airplane

In [27]:
v_plane = 250 * units.m / units.s  # East
phi = -26*units.deg

### Coriolis

In [28]:
f_26S = ofc.coriolis_parameter(phi)
f_26S

<Quantity -6.3933131e-05 1 / s>

In [29]:
a_plane = f_26S * v_plane
a_plane.to('m/s**2')

<Quantity -0.01598328 m / s2>

#### Direction is to the left in the Southern Hemisphere, i.e. North

In [31]:
# Deflection
x_coriolis = a_plane * (1*units.hour)**2 / 2
x_coriolis.to('km')

<Quantity -103.57167218 km>

### Compare

In [32]:
x_plane_E = v_plane * (1 * units.hour)
x_plane_E.to('km')

<Quantity 900. km>

#### Pretty substantial, i.e. nearly 10%

# 2) Rotating Tub

In [33]:
Dx = 7 * units.m
Dy = 8 * units.m

In [42]:
m = (10.22-10.13)/8.

In [43]:
def depth(y):
    h = 10.13*units.m + y*m
    return h

## a) Pressure gradient

### Let the bottom of the tub be $h=0$

In [38]:
rho = 1.

### In $x$

### $\partial p/\partial x = 0$

### In $y$  (note, the height of where we do this is not relevant)
 
### $\partial p/\partial y = (\partial h/\partial y) \rho_0 g = m \rho_0 g$

## b) Gradient at bottom

### There is no difference.

## c) Unfreeze

### Initial -- Fluid accelerates from high to low with acceleration

### $F_{\rm PG} = - \frac{1}{\rho_0} \frac{\partial p}{\partial y} = m g$

### Initial horizontal is due to Coriolis

### $F_C = f v(t=0) = 0$

# 3) Geostrophic Balance

## a) Definition

### In words, geostrophic balance is the balance of the horizontal component of the Coriolis force with the horizontal component of the pressure gradient force.

### The equations:

### $-fv = \frac{-1}{\rho_0} \frac{\partial p}{\partial x}$

### $fu = \frac{-1}{\rho_0} \frac{\partial p}{\partial y}$

### where $f$ is the Coriolis parameter, $u$ and $v$ are the $x$ and $y$ components of the velocity, $\rho_0$ is the  (assumed) constant density, and $p$ is the pressure.

## b) Estimated sea heights (from the figures)

In [49]:
def plot_heights(heights, title='Sea Height', color='blue'):
    # Plot
    plotting.output_notebook()

    p = plotting.figure(title=title, x_axis_label='x',
                                           y_axis_label='Sea Height (eta)')


    p.line(np.linspace(0.,1.,heights.size), heights, line_width=2 ,color=color)
    p.legend.location = "top_left"

    set_fontsize(p, 16)

    plotting.show(p)

### i) Diagonals galore

In [45]:
d_heights = np.array([0., 1., 2., 4., 8., 12., 16., 18., 19., 20.])
d_heights.size

10

In [50]:
plot_heights(d_heights, title='i) Diagonals')

Loading BokehJS ...

### ii) Circles High

In [51]:
H_heights = np.array([0., 1., 3., 7., 11., 13., 11., 7., 3., 1., 0.])

In [52]:
plot_heights(H_heights, title='ii) Circles High', color='green')

Loading BokehJS ...

### iii) Circles Low

In [53]:
L_heights = 13. - np.array([0., 1., 3., 7., 11., 13., 11., 7., 3., 1., 0.])

In [54]:
plot_heights(L_heights, title='iii) Circles Low', color='red')

Loading BokehJS ...

## c) Unfrozen (non-rotating)

In [88]:
arrows[1:-1]

[[0.9, 0.0, 1.5], [0.8, 0.0, 2], [0.7, 0.0, 3], [0.6, 0.0, 5]]

### i)

In [118]:
title = 'Diagonal'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ [1., 0., 1],
          [0.9, 0., 1.5],
          [0.8, 0., 2],
          [0.7, 0., 3],
          [0.6, 0., 5],
          [0.5, 0., 9],
          [0.4, 0., 15],
          [0.3, 0., 24],
          [0.2, 0., 36],
          [0.1, 0., 52],
          [0.0, 0., 76],
       ]
# Rest
for arrow in arrows[1:-1]:
    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

scale=4.
for arrow in arrows:
    arrow[2] = arrow[2]/scale

alen = 0.03
for arrow in arrows:
    asz = 5.
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=arrow[0], y_start=arrow[1], x_end=arrow[0]-alen, y_end=arrow[1]+alen, 
                              line_color='blue', line_width=arrow[2]))
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-arrow[1], y_start=1-arrow[0], x_end=1-arrow[1]-alen, y_end=1-arrow[0]+alen, 
                              line_color='blue', line_width=arrow[2]))
    # middle
    dx = 1.-arrow[0]
    #import pdb; pdb.set_trace()
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-dx/2, y_start=dx/2, x_end=1-dx/2-alen, y_end=dx/2+alen, 
                              line_color='blue', line_width=arrow[2]))
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-3*dx/4, y_start=dx/4, x_end=1-3*dx/4-alen, y_end=dx/4+alen, 
                              line_color='blue', line_width=arrow[2]))
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-dx/4, y_start=3*dx/4, x_end=1-dx/4-alen, y_end=3*dx/4+alen, 
                              line_color='blue', line_width=arrow[2]))


#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### ii) Circles High

In [107]:
title = 'Circles High'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ 
    [0.02, 0.8],
    [0.1, 1.],
          [0.2, 2],
          [0.3, 4],
          [0.4, 2],
          [0.5, 1],
       ]
# Rest
#for arrow in arrows[1:-1]:
#    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

#scale=2.
#for arrow in arrows:
#   arrow[2] = arrow[2]/scale

alen = 0.05
for arrow in arrows:
    for phi in np.linspace(0., 360., 10):
        x = 0.5 + arrow[0] * np.sin(phi*units.deg)
        y = 0.5 + arrow[0] * np.cos(phi*units.deg)
        dx = alen * np.sin(phi*units.deg)
        dy = alen * np.cos(phi*units.deg)
        #
        p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=2.5*arrow[1]),
                   x_start=x.value, y_start=y.value, x_end=(x+dx).value, y_end=(y+dy).value,                               
                      line_color='blue', line_width=arrow[1]))

#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### iii) Low

In [112]:
title = 'Circles Low'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ 
    #[0.02, 0.8],
    [0.1, 1.],
          [0.2, 2],
          [0.3, 4],
          [0.4, 2],
          [0.5, 1],
       ]
# Rest
#for arrow in arrows[1:-1]:
#    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

#scale=2.
#for arrow in arrows:
#   arrow[2] = arrow[2]/scale

alen = 0.05
for arrow in arrows:
    for phi in np.linspace(0., 360., 10):
        x = 0.5 + arrow[0] * np.sin(phi*units.deg)
        y = 0.5 + arrow[0] * np.cos(phi*units.deg)
        dx = -1*alen * np.sin(phi*units.deg)
        dy = -1*alen * np.cos(phi*units.deg)
        #
        p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=2.5*arrow[1]),
                   x_start=x.value, y_start=y.value, x_end=(x+dx).value, y_end=(y+dy).value,                               
                      line_color='blue', line_width=arrow[1]))

#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### If the system is allowed to reach equilibrium with viscosity, it will become a flat surface

## d) Unfreeze + rotation

### i) Diagonal

In [121]:
title = 'Diagonal Rotating'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ [1., 0., 1],
          [0.9, 0., 1.5],
          [0.8, 0., 2],
          [0.7, 0., 3],
          [0.6, 0., 5],
          [0.5, 0., 9],
          [0.4, 0., 15],
          [0.3, 0., 24],
          [0.2, 0., 36],
          [0.1, 0., 52],
          [0.0, 0., 76],
       ]
# Rest
for arrow in arrows[1:-1]:
    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

scale=4.
for arrow in arrows:
    arrow[2] = arrow[2]/scale

alen = 0.03
ayscl = 1.2
for arrow in arrows:
    asz = 5.
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=arrow[0], y_start=arrow[1], x_end=arrow[0]-alen, y_end=arrow[1]+alen*ayscl, 
                              line_color='blue', line_width=arrow[2]))
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-arrow[1], y_start=1-arrow[0], x_end=1-arrow[1]-alen, y_end=1-arrow[0]+alen*ayscl, 
                              line_color='blue', line_width=arrow[2]))
    # middle
    dx = 1.-arrow[0]
    #import pdb; pdb.set_trace()
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-dx/2, y_start=dx/2, x_end=1-dx/2-alen, y_end=dx/2+alen*ayscl, 
                              line_color='blue', line_width=arrow[2]))
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-3*dx/4, y_start=dx/4, x_end=1-3*dx/4-alen, y_end=dx/4+alen*ayscl, 
                              line_color='blue', line_width=arrow[2]))
    p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=asz),
                   x_start=1-dx/4, y_start=3*dx/4, x_end=1-dx/4-alen, y_end=3*dx/4+alen*ayscl, 
                              line_color='blue', line_width=arrow[2]))


#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### ii) High

In [115]:
title = 'Circles High Rotating'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ 
    [0.02, 0.8],
    [0.1, 1.],
          [0.2, 2],
          [0.3, 4],
          [0.4, 2],
          [0.5, 1],
       ]
# Rest
#for arrow in arrows[1:-1]:
#    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

#scale=2.
#for arrow in arrows:
#   arrow[2] = arrow[2]/scale

alen = 0.05
dphi = 10*units.deg
for arrow in arrows:
    for phi in np.linspace(0., 360., 10):
        x = 0.5 + arrow[0] * np.sin(phi*units.deg)
        y = 0.5 + arrow[0] * np.cos(phi*units.deg)
        dx = alen * np.sin(phi*units.deg+dphi)
        dy = alen * np.cos(phi*units.deg+dphi)
        #
        p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=2.5*arrow[1]),
                   x_start=x.value, y_start=y.value, x_end=(x+dx).value, y_end=(y+dy).value,                               
                      line_color='blue', line_width=arrow[1]))

#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### iii) Low

In [114]:
title = 'Circles Low Rotating'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ 
    #[0.02, 0.8],
    [0.1, 1.],
          [0.2, 2],
          [0.3, 4],
          [0.4, 2],
          [0.5, 1],
       ]
# Rest
#for arrow in arrows[1:-1]:
#    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

#scale=2.
#for arrow in arrows:
#   arrow[2] = arrow[2]/scale

alen = 0.05
dphi = 10*units.deg
for arrow in arrows:
    for phi in np.linspace(0., 360., 10):
        x = 0.5 + arrow[0] * np.sin(phi*units.deg)
        y = 0.5 + arrow[0] * np.cos(phi*units.deg)
        dx = -1*alen * np.sin(phi*units.deg+dphi)
        dy = -1*alen * np.cos(phi*units.deg+dphi)
        #
        p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=2.5*arrow[1]),
                   x_start=x.value, y_start=y.value, x_end=(x+dx).value, y_end=(y+dy).value,                               
                      line_color='blue', line_width=arrow[1]))

#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### My intuition tells me the equilibrium state for a circular bathtub is the same as the coffee cup exercise that we discussed in class.  Wait, that was the Fluids class.  Anyhow, the fluid will be a bit higher near the edge of the tub and lowest at the center.

## e) Southern tub

### My favorite

In [122]:
title = 'Circles Low Rotating Southern'
# Plot
plotting.output_notebook()

p = plotting.figure(title=title, x_axis_label='x', y_axis_label='y')

p.line(np.linspace(0.,1.,10), [0.]*10, line_width=2 ,color='black')

# Arrows
arrows = [ 
    #[0.02, 0.8],
    [0.1, 1.],
          [0.2, 2],
          [0.3, 4],
          [0.4, 2],
          [0.5, 1],
       ]
# Rest
#for arrow in arrows[1:-1]:
#    arrows.append([-1*arrow[0], arrow[1], arrow[2]])

#scale=2.
#for arrow in arrows:
#   arrow[2] = arrow[2]/scale

alen = 0.05
dphi = -10*units.deg
for arrow in arrows:
    for phi in np.linspace(0., 360., 10):
        x = 0.5 + arrow[0] * np.sin(phi*units.deg)
        y = 0.5 + arrow[0] * np.cos(phi*units.deg)
        dx = -1*alen * np.sin(phi*units.deg+dphi)
        dy = -1*alen * np.cos(phi*units.deg+dphi)
        #
        p.add_layout(models.Arrow(end=models.NormalHead(fill_color="blue", size=2.5*arrow[1]),
                   x_start=x.value, y_start=y.value, x_end=(x+dx).value, y_end=(y+dy).value,                               
                      line_color='blue', line_width=arrow[1]))

#
p.x_range = models.Range1d(0, 1)
p.y_range = models.Range1d(0, 1)


set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...

### Flow around high-pressure regions is anticyclonic.

### I think this is independent of Hemisphere (p. 31 of Talley Chapter 7 Supp)

----

# 4) Geostropic currents

----

# Tests

units.mile

In [ ]:
xxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
units.mile

# Tests